In [ ]:
from google.colab import files
uploaded = files.upload()

Saving category.py to category.py


In [ ]:
import boto3
import pandas as pd
from category import gif

#setup
CREDENTIALS_FILE = './credentials.csv'
credentials = pd.read_csv(CREDENTIALS_FILE).to_dict('records')[0]
aws_access_key_id = credentials['Access key ID']
aws_secret_access_key = credentials['Secret access key']

region_name = 'us-east-1'
endpoint_url = 'https://mturk-requester-sandbox.us-east-1.amazonaws.com'

client = boto3.client(
    'mturk',
    aws_access_key_id = aws_access_key_id,
    aws_secret_access_key = aws_secret_access_key,
    endpoint_url = endpoint_url,
    region_name = region_name
)


In [ ]:
#create qualification requirement

question = '''<QuestionForm xmlns='http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2005-10-01/QuestionForm.xsd'>
<Question>
      <QuestionIdentifier>password_test</QuestionIdentifier>
      <DisplayName>Question</DisplayName>
      <IsRequired>true</IsRequired>
      <QuestionContent>
        <Text> Please enter password </Text>
      </QuestionContent>
      <FreeTextAnswer>
        <Constraints>
          <Length minLength="2" />
          <AnswerFormatRegex regex="\S" errorText="The content cannot be blank."/>
        </Constraints>
      </FreeTextAnswer>
</Question> 
</QuestionForm>
'''

answer = '''<AnswerKey xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2005-10-01/AnswerKey.xsd">

</AnswerKey>
'''

qual_response = mturk.create_qualification_type(
                        Name='Enter password',
                        Keywords='test, qualification, password',
                        Description='Enter the correct password to access the questions',
                        QualificationTypeStatus='Active',
                        Test=question,
                        AnswerKey=answer,
                        TestDurationInSeconds=300)

print(qual_response['QualificationType']['QualificationTypeId'])

In [ ]:
#create hit type
one_minute = 60 # seconds
one_hour = 60 * one_minute
one_day = 24 * one_hour

hit_type_response = client.create_hit_type(
    AutoApprovalDelayInSeconds = 10 * one_minute,
    AssignmentDurationInSeconds = 5 * one_minute,
    Reward = '0.20',
    Title = 'Emotion in GIF',
    Keywords = 'emotion, GIF, detection',
    Description = 'Choose all emotions that you perceive from the GIFs',
    QualificationRequirements = [
      {
          'QualificationTypeId': '00000000000000000071',
          'Comparator': 'In',
          'LocaleValues': [
            {
                'Country': 'TW'
            },
            {
                'Country': 'US'
            },
          ],
          'RequiredToPreview': True,
          'ActionsGuarded': 'PreviewAndAccept'
      },
    ]
)

In [ ]:
hit_type_id = hit_type_response['HITTypeId']

for i in range(1, 44):
  cat = gif(i)
  cat_pro = cat.replace(' ', '%20')
  question = '''<?xml version="1.0" encoding="UTF-8"?>
  <ExternalQuestion xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2006-07-14/ExternalQuestion.xsd">
  '''
  question += "  <ExternalURL>https://jessicahuang523.github.io/emotionGIF/pages/"+cat_pro+".html</ExternalURL>"
  question += '''
    <FrameHeight>800</FrameHeight>
  </ExternalQuestion>
  '''

  response = client.create_hit_with_hit_type(
      HITTypeId = hit_type_id,
      MaxAssignments = 3,
      LifetimeInSeconds = 10 * one_minute,
      Question = question,
      RequesterAnnotation = cat
  )

  print(i, "done")


1 done
2 done
3 done
4 done
5 done
6 done
7 done
8 done
9 done
10 done
11 done
12 done
13 done
14 done
15 done
16 done
17 done
18 done
19 done
20 done
21 done
22 done
23 done
24 done
25 done
26 done
27 done
28 done
29 done
30 done
31 done
32 done
33 done
34 done
35 done
36 done
37 done
38 done
39 done
40 done
41 done
42 done
43 done


In [1]:
#fetch results

#list_hits_response = client.list_hits()
#hit_id = list_hits_response['HITs'][0]
#response = client.list_assignments_for_hit(HITId=hit_id)
hits_paginator = client.get_paginator('list_hits')
assignments_paginator = client.get_paginator('list_assignments_for_hit')
ans = ""
for hits in hits_paginator.paginate():
  for hit in hits['HITs']:
    for assignments in assignments_paginator.paginate(HITId=hit['HITId']):
      for assignment in assignments['Assignments']:
        print(assignment['Answer'])

NameError: ignored

In [ ]:
#extract answer
cat = ans.split("FreeText")
cat2 = cat[1].split("|")
for c in cat2:
  c = c.replace('>', '').replace('<', '').replace('/', '')
  print(c)